In [2]:
from data_common.db.duck import DuckQuery
from pathlib import Path
from data_common.pandas import GovLayers
import pandas as pd

In [4]:
# this is an alternate method of getting population
# basically using the onspd, we know the mapping of postcodes to lsoas, local authorities, and pcons
# using the lsoa population, we can work out the rough population of each postcode
# we then sum this back up for pcons, and for each overlap between pcons and local authorities

duck = DuckQuery()

onspd = Path("..", "data", "raw", "ONSPD_NOV_2022_UK_reduced.parquet")

lsoa_pop = Path("..", "data", "raw", "2019_population.csv")
lsoa_pop_df = pd.read_csv(lsoa_pop, thousands=",")


duck.register("onspd", onspd)
duck.register("lsoa_pop", lsoa_pop_df)

# get the count of how many postcodes in each lsoa

duck.query(
    "SELECT lsoa11 as lsoa, count(distinct pcd) as count FROM onspd group by all"
).to_view("lsoa_count").df().head()

# calculate average population per postcode - where this is 0, set to 1 (areas with high commerical, low residence, roughly this works out fine)
query = """
select
    lsoa,
    pop,
    count,
    case when cast(pop as float)/cast(count as float) = 0 then 1 else cast(pop as float)/cast(count as float) end as average_pop_per_postcode
from
    lsoa_count
join
    lsoa_pop using(lsoa)
"""

duck.query(query).to_view("lsoa_count_pop")

# sum up the average population per postcode for each pcon
query = """
select
    pcon,
    sum(average_pop_per_postcode) as pcon_pop
from
    onspd
join
    lsoa_count_pop on (onspd.lsoa11 = lsoa_count_pop.lsoa)
group by
    ALL
"""

duck.query(query).to_view("pcon_pop")

# this is the poplation overlap between the pcon and the la
query = """
select
    pcon,
    oslaua,
    sum(average_pop_per_postcode) as pop_overlap
from
    onspd
join
    lsoa_count_pop on (onspd.lsoa11 = lsoa_count_pop.lsoa)
group by
    ALL
order by
    pcon
"""

duck.query(query).to_view("pop_overlap")

# finally we can get the percentage of the pcon population that overlaps with the la

query = """
select
    pcon as PCON21CD,
    oslaua as LAD21CD,
    pop_overlap/pcon_pop as percentage_overlap
from
    pop_overlap
join
    pcon_pop using (pcon)
"""

df = duck.query(query).df()

df.to_csv(
    Path("..", "data", "interim", "percentage_overlap_2022_councils_pop.csv"),
    index=False,
)